In [1]:
import csv
import json
import sys
from typing import Any

import google.auth

from google.api_core.exceptions import AlreadyExists
from google.cloud import pubsub_v1
from google.cloud.pubsub_v1.types import BatchSettings

In [2]:
_, project_id = google.auth.default()
print(project_id)

/home/roberto/.conda/envs/aula-pdm/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
topic_id = "anuncios"
anuncios_file = "dados/anuncios-2025-09-08.csv"

In [ ]:
batch_settings = BatchSettings(
    max_bytes=5 * 1024 * 1024,  # No máximo 5MB por lote
    max_messages=1000,  # até 1000 messages por lote
    max_latency=0.25,  # envia a cada 250ms
)
publisher = pubsub_v1.PublisherClient(batch_settings=batch_settings)

In [ ]:
topic_path = publisher.topic_path(project_id, topic_id)
try:
    publisher.create_topic(name=topic_path)
except AlreadyExists:
    print(f"O tópico {topic_id} já existe.")

In [ ]:
int_fields = [
    "id_link",
    "anuncio_id",
    "qtd_quartos",
    "qtd_suites",
    "qtd_banheiros",
    "qtd_vagas_garagem",
    "id_anuncio_plataforma",
    "id_anuncio_externo",
    "id",
]
float_fields = [
    "area_total",
    "area_construida",
    "area_util",
    "preco",
    "iptu",
]
date_fields = [
    "data_cadastro",
    "data_atualizacao",
    "data_delecao",
    "data_criacao_anuncio_plataforma",
]


def enforce_types(row: dict[str, Any]) -> dict[str, Any]:
    for i in int_fields:
        if i in row and row[i] != "":
            try:
                row[i] = int(row[i])
            except ValueError:
                # print(f"WARNING: Could not convert field {i} to int: {row[i]}", file=sys.stderr)
                row[i] = None
        else:
            row[i] = None

    for f in float_fields:
        if f in row and row[f] != "":
            try:
                row[f] = float(row[f])
            except ValueError:
                # print(f"WARNING: Could not convert field {f} to float: {row[f]}", file=sys.stderr)
                row[f] = None
        else:
            row[f] = None

    for d in date_fields:
        if d in row and row[d] == "":
            row[d] = None

    return row

In [3]:
# Lê o CSV e armazena os JSONs convertidos em uma lista
linhas_json = []
num_rows = 0
with open(anuncios_file, "r", encoding="utf-8", newline="") as f:
    reader = csv.DictReader(f, delimiter=",")
    if reader.fieldnames is None:
        print("ERROR: CSV appears to have no header row.", file=sys.stderr)
        sys.exit(2)

    for idx, row in enumerate(reader, start=1):
        # Converte a linha do CSV para JSON
        linha_json = json.dumps(enforce_types(row), ensure_ascii=False)
        linhas_json.append(linha_json)
        num_rows += 1

print(f"Foram lidas {num_rows} linhas do arquivo {anuncios_file}.")

In [5]:
print(linhas_json[0])

{"site": "olx", "id_link": "39", "anuncio_id": "19", "descricao": "Código do anúncio: 885-15_2-3721005Contato // Whatsapp: (62) 9.8208.8690 Apartamento Mobiliado à Venda Jardim América Pronto para morar! Mobiliado, climatizado e com excelente custo-benefício. - 62 m² - 2 Quartos (1 suíte com ar-condicionado) - Ar-condicionado também na sala - 2 Vagas de garagem - Posição solar transversal - Andar baixo - R 490.000 - Condomínio: R 590,00 Lazer completo: - Piscina - Espaço gourmet - Playground - Mini mercado no condomínio Perfeito para quem busca conforto, praticidade e localização estratégica! (62) 9.8208.8690 Danilo Oliveira (62) 9.9982.4360 Ellen Cordeiro", "titulo": "Apartamento para Venda em Goiânia, Jardim América, 2 dormitórios, 1 suíte, 2 banheiros, 2", "endereco_completo": "Jardim América, Goiânia, GO", "cep": "74265050", "andar": "", "area_total": "", "area_construida": "", "area_util": "62.0", "qtd_quartos": "2", "qtd_suites": "", "qtd_banheiros": "2", "qtd_vagas_garagem": "2"

In [ ]:
futures = []
for idx, linha in enumerate(linhas_json):
    future = publisher.publish(topic_path, data=linha.encode("utf-8"), row=str(idx))
    futures.append(future)

In [ ]:
for fut in futures:
    fut.result(timeout=60)